In [ ]:
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth

In [ ]:
movie_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movies.csv',usecols=['movieId','title'], dtype={'movieId':'int32','title':'str'})

In [ ]:
rating_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ratings.csv',usecols=['userId','movieId','rating'],dtype={'userId':'int32','movieId':'int32','rating':'float32'})

In [ ]:
movie_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [ ]:
movie_df.shape

(9742, 2)

In [ ]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.000
1,1,3,4.000
2,1,6,4.000
3,1,47,5.000
4,1,50,5.000


In [ ]:
rating_df.shape

(100836, 3)

In [ ]:
df = pd.merge(rating_df,movie_df, on='movieId')
df

,userId,movieId,rating,title
0,1,1,4.000,Toy Story (1995)
1,5,1,4.000,Toy Story (1995)
2,7,1,4.500,Toy Story (1995)
3,15,1,2.500,Toy Story (1995)
4,17,1,4.500,Toy Story (1995)
...,...,...,...,...
100831,610,160341,2.500,Bloodmoon (1997)
100832,610,160527,4.500,Sympathy for the Underdog (1971)
100833,610,160836,3.000,Hazard (2005)
100834,610,163937,3.500,Blair Witch (2016)


In [ ]:
combine_movie_rating = df.dropna(axis = 0, subset = ['title'])
movie_ratingCount = (combine_movie_rating.groupby(by = ['title'])['rating'].count().reset_index().rename(columns = {'rating':'totalRatingCount'})[['title','totalRatingCount']])
movie_ratingCount

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2
...,...,...
9714,eXistenZ (1999),22
9715,xXx (2002),24
9716,xXx: State of the Union (2005),5
9717,¡Three Amigos! (1986),26


In [ ]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on = 'title', right_on ='title', how = 'left')
rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.000,Toy Story (1995),215
1,5,1,4.000,Toy Story (1995),215
2,7,1,4.500,Toy Story (1995),215
3,15,1,2.500,Toy Story (1995),215
4,17,1,4.500,Toy Story (1995),215


In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(movie_ratingCount['totalRatingCount'].describe())

count   9719.000
mean      10.375
std       22.406
min        1.000
25%        1.000
50%        3.000
75%        9.000
max      329.000
Name: totalRatingCount, dtype: float64


In [ ]:
popularity_treshold = 85
rating_popular_movie = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_treshold')
rating_popular_movie.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.000,Toy Story (1995),215
1,5,1,4.000,Toy Story (1995),215
2,7,1,4.500,Toy Story (1995),215
3,15,1,2.500,Toy Story (1995),215
4,17,1,4.500,Toy Story (1995),215


In [ ]:
rating_popular_movie.shape

(24849, 5)

In [ ]:
## pivot matrix

movie_features_df = rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
Ace Ventura: Pet Detective (1994),0.000,0.000,0.000,0.000,3.000,3.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,2.000,0.000,0.000,0.000,3.500,0.000,3.000
Ace Ventura: When Nature Calls (1995),0.000,0.000,0.000,0.000,0.000,2.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,1.000,0.000,2.000,0.000,2.000,0.000,0.000
Airplane! (1980),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,5.000,0.000,4.000,0.000,0.000,3.500
Aladdin (1992),0.000,0.000,0.000,4.000,4.000,5.000,3.000,0.000,0.000,4.000,...,0.000,0.000,0.000,3.000,3.500,0.000,0.000,3.000,0.000,0.000


In [ ]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_features_df.values)

from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
Ace Ventura: Pet Detective (1994),0.000,0.000,0.000,0.000,3.000,3.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,2.000,0.000,0.000,0.000,3.500,0.000,3.000
Ace Ventura: When Nature Calls (1995),0.000,0.000,0.000,0.000,0.000,2.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,1.000,0.000,2.000,0.000,2.000,0.000,0.000
Airplane! (1980),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,5.000,0.000,4.000,0.000,0.000,3.500
Aladdin (1992),0.000,0.000,0.000,4.000,4.000,5.000,3.000,0.000,0.000,4.000,...,0.000,0.000,0.000,3.000,3.500,0.000,0.000,3.000,0.000,0.000


In [ ]:
query_index = np.random.choice(movie_features_df.shape[0])
print(query_index)
distance, indices = model_knn.kneighbors(movie_features_df.iloc[query_index, :].values.reshape(1, -1),n_neighbors=100)

180


In [ ]:
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
Ace Ventura: Pet Detective (1994),0.000,0.000,0.000,0.000,3.000,3.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,2.000,0.000,0.000,0.000,3.500,0.000,3.000
Ace Ventura: When Nature Calls (1995),0.000,0.000,0.000,0.000,0.000,2.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,1.000,0.000,2.000,0.000,2.000,0.000,0.000
Airplane! (1980),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,5.000,0.000,4.000,0.000,0.000,3.500
Aladdin (1992),0.000,0.000,0.000,4.000,4.000,5.000,3.000,0.000,0.000,4.000,...,0.000,0.000,0.000,3.000,3.500,0.000,0.000,3.000,0.000,0.000


In [ ]:
for i in range(0, len(distance.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_features_df.index[query_index]))
    else:
        print('{0}:{1}, with distance of {2}:'.format(
            i, movie_features_df.index[indices.flatten()[i]], distance.flatten()[i]))

Recommendations for V for Vendetta (2006):

1:Batman Begins (2005), with distance of 0.3686490058898926:
2:Kill Bill: Vol. 1 (2003), with distance of 0.3965873122215271:
3:Kill Bill: Vol. 2 (2004), with distance of 0.40520524978637695:
4:Pirates of the Caribbean: The Curse of the Black Pearl (2003), with distance of 0.41426175832748413:
5:Departed, The (2006), with distance of 0.4178907871246338:
6:Lord of the Rings: The Fellowship of the Ring, The (2001), with distance of 0.42669230699539185:
7:Prestige, The (2006), with distance of 0.4292562007904053:
8:Lord of the Rings: The Return of the King, The (2003), with distance of 0.4374404549598694:
9:Incredibles, The (2004), with distance of 0.44701921939849854:
10:Fight Club (1999), with distance of 0.45029616355895996:
11:Memento (2000), with distance of 0.45365655422210693:
12:Lord of the Rings: The Two Towers, The (2002), with distance of 0.45433783531188965:
13:Matrix Reloaded, The (2003), with distance of 0.4607490301132202:
14:Ocea